In [12]:
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
import alpaca_trade_api as tradeapi
import pandas as pd

ModuleNotFoundError: No module named 'alpaca_trade_api'

In [ ]:
apiKey = 'AKREF539HEBKQQ6OID3U'
secret = 'q1ysWDHYs1oZVmxDqtIjmV9E4pMS8aEhobw7ihl0'

# API credentials
API_KEY = "your_api_key"
API_SECRET = "your_api_secret"
BASE_URL = "https://paper-api.alpaca.markets"  # Use this for paper trading

# Initialize API
api = tradeapi.REST(API_KEY, API_SECRET, BASE_URL, api_version='v2')

# Get historical data
barset = api.get_bars("AAPL", "day", limit=5)  # 'minute', 'hour', or 'day'
df = pd.DataFrame([bar.__dict__['_raw'] for bar in barset])

print(df)


NameError: name 'apiKey' is not defined